In [3]:
import pandas as pd
import numpy as np
import openpyxl
import json

In [4]:
# 读取xlsx文件
workbook = openpyxl.load_workbook('../culture/res.xlsx')
sheet = workbook.active

# 读取json文件
with open('all_trees.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 遍历json里每一个对象
for obj in data:
    template_id = obj['template']['id']
  
    # 在xlsx里找对应的行
    for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=5):
        if row[0].value == template_id:
            key_phrase = row[4].value
            if(key_phrase is None):
                obj['key_phrase'] = "暂无"
            else:
              obj['key_phrase'] = key_phrase
            break

# 新增领养情况字段
for obj in data:
    if obj.get('owner') is None:
        obj['hasOwner'] = "暂无领养人"
        obj['ownerTime'] = "暂无"
    else:
        obj['hasOwner'] = "已被来自" + obj['owner']['title'] + "的校友领养"
        obj['ownerTime'] = obj['owner']['updatedAt']

# 对字符串和null做替换
def replace_null_and_empty(obj):
    for key, value in obj.items():
        if value == "":
            obj[key] = "暂无"
        elif value is None:
            obj[key] = "暂无"
        elif isinstance(value, dict):
            replace_null_and_empty(value)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    replace_null_and_empty(item)

replace_null_and_empty(obj)

# 写回json文件
with open('all_trees_for_lib.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)